# Code inspired from article https://towardsdatascience.com/text-summarization-with-nlp-textrank-vs-seq2seq-vs-bart-474943efeb09  

# import libraries

In [14]:
## for data
import datasets 
import pandas as pd 
import numpy 
## for plotting
import matplotlib.pyplot as plt 
import seaborn as sns 
## for preprocessing
import re
import nltk 
import contractions 
import os
## for textrank
import gensim #summarize was deleted in later version due to poor performance, never in use in Production, not maintained
import pytextrank
import spacy
## for evaluation
import rouge  
import difflib
## for seq2seq
from tensorflow.keras import callbacks, models, layers, preprocessing as kprocessing 
## for bart
#import transformers
##for generating sentence after textrank 
import openai
from transformers import T5Tokenizer, T5ForConditionalGeneration

ImportError: cannot import name 'TFT5Tokenizer' from 'transformers' (C:\Users\loren\.virtualenvs\text_summarization-RZoufpZG\lib\site-packages\transformers\__init__.py)

# import dataset 

In [2]:
## load the full dataset of 300k articles
dataset = datasets.load_dataset("cnn_dailymail", '3.0.0')
lst_dics = [dic for dic in dataset["train"]]

Found cached dataset cnn_dailymail (C:/Users/loren/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
## keep the first N articles if you want to keep it lite 
dtf = pd.DataFrame(lst_dics).rename(columns={"article":"text", 
      "highlights":"y"})[["text","y"]].head(20000)
dtf.head()

,text,y
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa..."
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non..."
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical..."


In [4]:
#check one example 
i = 0
print("--- Full text ---")
print(dtf["text"][i])
print("--- Summary ---")
print(dtf["y"][i])

--- Full text ---
LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office cha

# create train/test dataset

In [5]:
dtf_train = dtf.iloc[i+1:]
dtf_test = dtf.iloc[:i+1]

# TextRank algorithm

In [6]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.1/12.8 MB 1.7 MB/s eta 0:00:08
     ---------------------------------------- 0.1/12.8 MB 1.3 MB/s eta 0:00:10
     ---------------------------------------- 0.1/12.8 MB 1.0 MB/s eta 0:00:13
     --------------------------------------- 0.1/12.8 MB 944.1 kB/s eta 0:00:14
      --------------------------------------- 0.2/12.8 MB 1.0 MB/s eta 0:00:13
      --------------------------------------- 0.2/12.8 MB 1.0 MB/s eta 0:00:13
      --------------------------------------- 0.3/12.8 MB 1.0 MB/s eta 0:00:13
     - -------------------------------------- 0.3/12.8 MB 1.1 MB/s eta 0:00:12
     - -------------------------------------- 0.4/12.8 MB 1.1 MB/s eta 0:00:12
     - -------------------------------------- 0.4/12.8 MB 1.1 MB/s eta 0:00:12
     - -------------------------------------- 0.5/12.8 MB 1.2 MB/s eta 0:00:11
     - -------------------------------------- 0.5/12.8 MB 

In [7]:
# Load a spaCy model
nlp = spacy.load("en_core_web_sm")

# Add PyTextRank to the spaCy pipeline
nlp.add_pipe("textrank")

In [8]:
'''
GENSIM OUTDATED VERSION
Summarizes corpus with TextRank.
:parameter    
    :param corpus: str or list - dtf["text"]    
    :param ratio: length of the summary (ex. 20% of the text)
:return    
    list of summaries
'''
def textrank(corpus, ratio=0.2):    
    if type(corpus) is str:        
        corpus = [corpus]    
    lst_summaries = [gensim.summarization.summarize(txt,  
                     ratio=ratio) for txt in corpus]    
    return lst_summaries

In [9]:
# Define a function to apply textrank algorithm to corpus with a ratio parameter
def ptrank(corpus, ratio=0.2):
    if type(corpus) is str:
        corpus = [corpus]
    lst_phrases = []
    for txt in corpus:
        # Parse the document with spaCy
        doc = nlp(txt)
        # Extract the top-ranked phrases from the document
        phrases = []
        for phrase in doc._.phrases:
            phrases.append((phrase.text, phrase.rank))
        # Sort the phrases by rank
        phrases = sorted(phrases, key=lambda x: x[1], reverse=True)
        # Limit the number of phrases based on the ratio
        limit = int(len(phrases) * ratio)
        # Append only the top phrases to the lst_phrases list
        lst_phrases.append(phrases[:limit])
    return lst_phrases

In [10]:
# Apply the function to corpus with a ratio of 0.2
predicted  = ptrank(corpus=dtf_test["text"], ratio=0.2)
predicted [i]

[('last month', 0.09028334388935466),
 ('Harry Potter', 0.08470743888327945),
 ('Harry Potter star Daniel Radcliffe', 0.08451745312035847),
 ('Potter', 0.07130478022388759),
 ('celebrity parties', 0.06502224902268608),
 ('fast cars', 0.06323968373052252),
 ('Part II', 0.06025288049638938),
 ('author Rudyard Kipling', 0.059676352987717604),
 ('party', 0.05490785094705813),
 ('Rudyard Kipling', 0.05388817654532417),
 ('kid star', 0.05344539230182263),
 ('drink', 0.051234195299302315),
 ('the UK box office chart', 0.050236193717111216),
 ('Reuters', 0.04823424253485198),
 ('DVDs', 0.04749590035052297),
 ('Daniel Radcliffe', 0.04741469242892149),
 ('Phoenix', 0.04683664224799264),
 ('fair game', 0.04682495302054514),
 ('release', 0.046285759234409585),
 ('Australian', 0.04606602314363066),
 ('Atlantic', 0.044793935362700106),
 ('UK', 0.04444162895327798),
 ('the horror film', 0.04431792905585396),
 ('records', 0.04381254489108988),
 ('Potters latest »', 0.04372498306327185),
 ('the first f

## Result using basic concatenation of top[ratio] words 

In [11]:
s = ",".join([str(t[0]) for t in predicted[i]])
print(s)

last month,Harry Potter,Harry Potter star Daniel Radcliffe,Potter,celebrity parties,fast cars,Part II,author Rudyard Kipling,party,Rudyard Kipling,kid star,drink,the UK box office chart,Reuters,DVDs,Daniel Radcliffe,Phoenix,fair game,release,Australian,Atlantic,UK,the horror film,records,Potters latest »,the first five Potter films,a massive sports car collection


## Result using keyword and OPEN API GPT3 Davinci (billing problem for now, contacted openAI)

In [59]:
with open("open_ai_api_key.txt", "r") as f:
    openai.api_key = f.read().rstrip()

In [60]:
separator = ","

In [61]:
# Format predicted as a partial text
input = "Generate a sentence from the following keywords:\n\n"
for keyword, value in predicted[i]:
    input += f"- {keyword}, {value}\n"
input += "\nSentence:"

In [64]:
# Send a request to the OpenAI API using the text completion feature
response = openai.Completion.create(
    engine="davinci",
    prompt=input,
    max_tokens=50,
    temperature=0.5,
    frequency_penalty=0.5,
)

RateLimitError: You exceeded your current quota, please check your plan and billing details.

In [55]:
# Print the generated sentence
print(response["choices"][0]["text"])

NameError: name 'response' is not defined

## Same using HuggingFace transformers

In [13]:
# Load tokenizer and model
tokenizer = TFT5Tokenizer.from_pretrained("t5-base")
model = TFT5ForConditionalGeneration.from_pretrained("t5-base")

NameError: name 'TFT5Tokenizer' is not defined

# Evaluate result 

### ROUGE (Recall-Oriented Understudy for Gisting Evaluation) score 

In [29]:
!pip list

Package                      Version
---------------------------- ---------
absl-py                      1.4.0
aiohttp                      3.8.4
aiosignal                    1.3.1
anyascii                     0.3.2
anyio                        3.6.2
argon2-cffi                  21.3.0
argon2-cffi-bindings         21.2.0
arrow                        1.2.3
asttokens                    2.2.1
astunparse                   1.6.3
async-timeout                4.0.2
attrs                        23.1.0
backcall                     0.2.0
beautifulsoup4               4.12.2
bleach                       6.0.0
blis                         0.7.9
cachetools                   5.3.0
catalogue                    2.0.8
certifi                      2022.12.7
cffi                         1.15.1
charset-normalizer           3.1.0
click                        8.1.3
colorama                     0.4.6
comm                         0.1.3
confection                   0.0.4
contourpy                    1.0.7
contr

In [30]:
'''
Calculate ROUGE score.
:parameter    
    :param y_test: string or list    
    :param predicted: string or list
'''
def evaluate_summary(y_test, predicted):    
    rouge_score = rouge.Rouge()
    # join the elements of the lists with a space character
    print(y_test)
    print(predicted)
    scores = rouge_score.get_scores(y_test, predicted, avg=True)       
    score_1 = round(scores['rouge-1']['f'], 2)    
    score_2 = round(scores['rouge-2']['f'], 2)    
    score_L = round(scores['rouge-l']['f'], 2)    
    print("rouge1:", score_1, "| rouge2:", score_2, "| rougeL:",
         score_2, "--> avg rouge:", round(np.mean(
         [score_1,score_2,score_L]), 2))

In [31]:
## Apply the function to predicted
evaluate_summary(dtf_test["y"][i], predicted[i])

Parents beam with pride, can't stop from smiling from outpouring of support .
Mom: "I was so happy I didn't know what to do"
Burn center in U.S. has offered to provide treatment for reconstructive surgeries .
Dad says, "Anything for Youssif"
[('Youssif', 0.08991604280608116), ('burn victims', 0.07712531650502016), ('next Friday', 0.06321945993269831), ('Grossman Burn Center', 0.06265300861269642), ('victims', 0.059089448141081316), ('January day', 0.05883528589972477), ('Friday', 0.05709053619891388), ('Youssifs story', 0.056738529916598335), ('donations', 0.05573868660201247), ('support', 0.05540512809334803), ('people', 0.05451055206641087), ('Sherman Oaks', 0.05397623955921135), ('housing costs', 0.05076959110171379), ('Iraq', 0.050019070383050056), ('Youssifs cause', 0.04841783443181997), ('Youssifs case', 0.04725255342337501), ('Dr. Peter Grossman', 0.04544928150323525), ('CNN Iraqi staff', 0.04499074017723958), ('help', 0.04483858989149564), ('Peter Grossman', 0.04419040566287786

AttributeError: 'list' object has no attribute 'split'

### Visualization 

In [8]:
'''
Find the matching substrings in 2 strings.
:parameter
    :param a: string - raw text
    :param b: string - raw text
:return
    2 lists used in to display matches
'''
def utils_split_sentences(a, b):
    ## find clean matches
    match = difflib.SequenceMatcher(isjunk=None, a=a, b=b, autojunk=True)
    lst_match = [block for block in match.get_matching_blocks() if block.size > 20]
    
    ## difflib didn't find any match
    if len(lst_match) == 0:
        lst_a, lst_b = nltk.sent_tokenize(a), nltk.sent_tokenize(b)
    
    ## work with matches
    else:
        first_m, last_m = lst_match[0], lst_match[-1]

        ### a
        string = a[0 : first_m.a]
        lst_a = [t for t in nltk.sent_tokenize(string)]
        for n in range(len(lst_match)):
            m = lst_match[n]
            string = a[m.a : m.a+m.size]
            lst_a.append(string)
            if n+1 < len(lst_match):
                next_m = lst_match[n+1]
                string = a[m.a+m.size : next_m.a]
                lst_a = lst_a + [t for t in nltk.sent_tokenize(string)]
            else:
                break
        string = a[last_m.a+last_m.size :]
        lst_a = lst_a + [t for t in nltk.sent_tokenize(string)]

        ### b
        string = b[0 : first_m.b]
        lst_b = [t for t in nltk.sent_tokenize(string)]
        for n in range(len(lst_match)):
            m = lst_match[n]
            string = b[m.b : m.b+m.size]
            lst_b.append(string)
            if n+1 < len(lst_match):
                next_m = lst_match[n+1]
                string = b[m.b+m.size : next_m.b]
                lst_b = lst_b + [t for t in nltk.sent_tokenize(string)]
            else:
                break
        string = b[last_m.b+last_m.size :]
        lst_b = lst_b + [t for t in nltk.sent_tokenize(string)]
    
    return lst_a, lst_b

In [12]:
'''
Highlights the matched strings in text.
:parameter
    :param a: string - raw text
    :param b: string - raw text
    :param both: bool - search a in b and, if True, viceversa
    :param sentences: bool - if False matches single words
:return
    text html, it can be visualized on notebook with display(HTML(text))
'''
def display_string_matching(a, b, both=True, sentences=True, titles=[]):
    if sentences is True:
        lst_a, lst_b = utils_split_sentences(a, b)
    else:
        lst_a, lst_b = a.split(), b.split()       
    
    ## highlight a
    first_text = []
    for i in lst_a:
        if re.sub(r'[^\w\s]', '', i.lower()) in [re.sub(r'[^\w\s]', '', z.lower()) for z in lst_b]:
            first_text.append('<span style="background-color:rgba(255,215,0,0.3);">' + i + '</span>')
        else:
            first_text.append(i)
    first_text = ' '.join(first_text)
    
    ## highlight b
    second_text = []
    if both is True:
        for i in lst_b:
            if re.sub(r'[^\w\s]', '', i.lower()) in [re.sub(r'[^\w\s]', '', z.lower()) for z in lst_a]:
                second_text.append('<span style="background-color:rgba(255,215,0,0.3);">' + i + '</span>')
            else:
                second_text.append(i)
    else:
        second_text.append(b) 
    second_text = ' '.join(second_text)
    
    ## concatenate
    if len(titles) > 0:
        first_text = "<strong>"+titles[0]+"</strong><br>"+first_text
    if len(titles) > 1:
        second_text = "<strong>"+titles[1]+"</strong><br>"+second_text
    else:
        second_text = "---"*65+"<br><br>"+second_text
    final_text = first_text +'<br><br>'+ second_text
    return final_text